# Base de la Tarea dada en el enunciado

Antes de empezar es necesario ver lo que nos dieron en el enunciado para poder crear las funciones necesarias:


La primera es davies-meyer, la funcion de compresión, que debe retornar una funcion de compresion basada en las construccion que vimos en clases

In [1]:
# ( encrypt : ( bytearray , bytearray ) - > bytearray ,l_key : int , l_message : int )- > ( bytearray ) - > bytearray :
def davies_meyer(encrypt, l_key, l_message):
    """
    Arguments :
        encrypt : an encryption function
        l_key : length in bytes of the keys for encrypt
        l_message : length in bytes of the messages for encrypt
    Returns :
        A compression function from messages of length l_key + l_message to
        messages of length l_message , defined by using the Davies - Meyer
        construction
    """
    pass

La segunda funcion que nos piden es pad, la cual en mi opinion es la más simple y más adelante será la primera en ser desarrollada


In [2]:
def pad(message, l_block):
    """
    Arguments :
        message : message to be padded
        l_block : length in bytes of the block
    Returns :
        extension of message that includes the length of message
        ( in bytes ) in its last block
    """
    pass

Por ultimo la funcion de merkle-damgard, que hace uso de las dos anteriores por lo que será la final y que tambien retorna una funcion de hash.


In [3]:
#( IV : bytearray , comp : ( bytearray ) - > bytearray ,
#l_block : int ) - > ( bytearray ) - > bytearray :
def merkle_damgard(IV, comp, l_block):
    """
    Arguments :
        IV : initialization vector for a hash function
        comp : compression function to be used in the Merkle - Damgard
        construction
        l_block : length in bytes of the blocks to be used in the Merkle - Damgard
        construction
    Returns :
        A hash function for messages of arbitrary length , defined by using
        the Merkle - Damgard construction
    """
    pass

# Funcion Pad

Basta con seguir las indicaciones que nos dieron en un inicio de volver al mensaje en uno divisible por el largo de los bloques y agregar un bloque al final que indique el largo de los datos en base 256

In [23]:
def pad(message, l_block):
    length = len(message)
    remainder = l_block - length%l_block
    if remainder == l_block:
        # En caso de ser divisible solo se debe agregar el largo
        padded = message + length.to_bytes(l_block, "big")
        
    else: 
        # En caso contrario se agregan los bytes \x01\x00...\x00
        if remainder == 1:
            # Solo agrego 1 y ningun \x00
            padded = message + b'\x01' + length.to_bytes(l_block, "big")
            
        else:
            # Agrego un \x01 y remainder-1 veces \x00
            padded = message + b'\x01' + b'\x00'*(remainder-1) + length.to_bytes(l_block, "big")
    return padded

Para comprobar pruebo con los ejemplos que nos dieron en el git en el area de issues

In [24]:
pad(b'0', 16) == b'0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01'

True

In [25]:
pad(b'0123456789012345', 16) == b'0123456789012345\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10'

True

In [26]:
padded = pad(b'012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789', 16)
issue = b'012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01,'

padded==issue

True

# Compresión por construcción de Davies-Meyer

En base a lo visto en clases se desarrolla esta funcion

In [19]:
def davies_meyer(encrypt, l_key, l_message):
    # Primero defino una funcion de xor bit a bit
    def byte_xor(ba1, ba2):
        return bytearray([_a ^ _b for _a, _b in zip(ba1, ba2)])
    
    def comp(message):
        n_key = message[:l_key]
        n_mess = message[l_key:l_key+l_message]
        
        compressed = byte_xor(encrypt(n_key, n_mess), n_mess)
        
        return compressed
    return comp

In [48]:
from Crypto.Cipher import AES

def AES_128(key: bytearray, message: bytearray) -> bytearray:
    a = AES.new(bytes(key), AES.MODE_ECB)
    return bytearray(a.encrypt(message))

# compr = davies_meyer(AES_128, 16, 16)

Para comprobar que al menos este comprimiendo hago una prueba con un mensaje de largo 32 `(l_key + l_message)`

In [44]:
# compr(bytearray(b"123456789012345678901234567890123456"))

TypeError: argument 1 must be read-only bytes-like object, not bytearray

Para evitar este tipo de errores en la construccion de Merkle-Damgard se antepone un "bytes" a el mensaje a encriptar

# Construccion de Merkle-Damgard
Finalmente se aplica el modelo de bloques para realizar la encriptacion por partes que corresponde a este tipo de funciones

In [45]:
def merkle_damgard(IV, comp, l_block):
    def pad(message, l_block):
        length = len(message)
        remainder = l_block - length%l_block
        if remainder == l_block:
            # En caso de ser divisible solo se debe agregar el largo
            padded = message + length.to_bytes(l_block, "big")
        
        else: 
            # En caso contrario se agregan los bytes \x01\x00...\x00
            if remainder == 1:
                # Solo agrego 1 y ningun \x00
                padded = message + b'\x01' + length.to_bytes(l_block, "big")
            
            else:
                # Agrego un \x01 y remainder-1 veces \x00
                padded = message + b'\x01' + b'\x00'*(remainder-1) + length.to_bytes(l_block, "big")
        return padded

    def hash_function(message):
        message = pad(message, l_block)
        
        H = IV
        for i in range(0,len(message), l_block):
            # print(message[i:i+l_block])
            H = comp(bytes(message[i:i+l_block]+H))
            
        return H
        
    
    return hash_function

In [46]:
#hashing = merkle_damgard(bytearray(b'1234567890123456'), compr, 16)
#hashing(bytearray(b''))

bytearray(b'p\xca \xd8\x9c\xeb\xe6\xb1\xce\xcf\x03\xb2\x9e\x93\x19\xbc')

In [49]:
if __name__ == "__main__":
    compresion = davies_meyer(AES_128, 16, 16)
    hash = merkle_damgard(bytearray(b'1234567890123456'), compresion, 16)

    s1 = bytearray(b'Este es un mensaje de prueba para la tarea 2')
    s2 = bytearray(b'Este es un mensaje de Prueba para la tarea 2')
    s3 = bytearray(b'Un mensaje corto')
    s4 = bytearray(b'')

    h1 = hash(s1)
    h2 = hash(s2)
    h3 = hash(s3)
    h4 = hash(s4)

    print(h1)
    print(h2)
    print(h3)
    print(h4)

bytearray(b'\xe9\xe8\xac\x12\\\xf2\xc8\x16\xceOV\xc5Y.T\xea')
bytearray(b'\xb6\xfb\xc6a\x12\xae\x95\x1f\xda\xc5\x13\xde\x06|Q\x96')
bytearray(b'\xc5\xec\xcdd\xa4(R*\xf0L*QtL\xda\x81')
bytearray(b'p\xca \xd8\x9c\xeb\xe6\xb1\xce\xcf\x03\xb2\x9e\x93\x19\xbc')
